In [1]:
import pandas as pd
from pathlib import Path
import numpy as np

source_folder = Path("atp_transformed/2000-2024 with covid.csv")

In [2]:
df = pd.read_csv(source_folder) 
df

C:\Users\Golbe\AppData\Local\Temp\ipykernel_15972\535734251.py:1: DtypeWarning: Columns (5,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(source_folder)


,tourney_id,tourney_name,tourney_type,surface,draw_size,tourney_level,tourney_date,match_num,score,round_1,...,double_faults,points_on_serve,first_serve_in,1st_won,2nd_won,service_games,break_points_saved,break_points_faced,match_outcome,player_tourney_match_id
0,2000-301,Auckland,matches,Hard,32,A,2000-01-10,1,7-5 4-6 7-5,7-5,...,4.0,96.0,49.0,39.0,28.0,17.0,3.0,5.0,1,103163:2000-301:1
1,2000-301,Auckland,matches,Hard,32,A,2000-01-10,1,7-5 4-6 7-5,7-5,...,4.0,96.0,49.0,39.0,28.0,17.0,3.0,5.0,0,101543:2000-301:1
2,2000-301,Auckland,matches,Hard,32,A,2000-01-10,2,7-5 7-5,7-5,...,3.0,76.0,52.0,39.0,13.0,12.0,5.0,6.0,1,102607:2000-301:2
3,2000-301,Auckland,matches,Hard,32,A,2000-01-10,2,7-5 7-5,7-5,...,3.0,76.0,52.0,39.0,13.0,12.0,5.0,6.0,0,102644:2000-301:2
4,2000-301,Auckland,matches,Hard,32,A,2000-01-10,3,6-3 6-1,6-3,...,0.0,55.0,35.0,25.0,12.0,8.0,1.0,1.0,1,103252:2000-301:3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1289975,2024-M-ITF-UZB-2024-002,M15 Tashkent,futures,Hard,32,15,2024-09-02,401,6-4 3-6 6-3,6-4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,210011:2024-M-ITF-UZB-2024-002:401
1289976,2024-M-ITF-UZB-2024-002,M15 Tashkent,futures,Hard,32,15,2024-09-02,402,6-2 3-6 6-2,6-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,209951:2024-M-ITF-UZB-2024-002:402
1289977,2024-M-ITF-UZB-2024-002,M15 Tashkent,futures,Hard,32,15,2024-09-02,402,6-2 3-6 6-2,6-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,106168:2024-M-ITF-UZB-2024-002:402
1289978,2024-M-ITF-UZB-2024-002,M15 Tashkent,futures,Hard,32,15,2024-09-02,501,4-6 6-1 6-4,4-6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,209951:2024-M-ITF-UZB-2024-002:501


In [3]:
# Ensure tournament_date is datetime
df["tourney_date"] = pd.to_datetime(df["tourney_date"])

# Extract tournament year
df["year"] = df["tourney_date"].dt.year

# Find each player's first tournament date
first_tournament = (
    df.groupby("player_id")["tourney_date"]
      .min()
      .rename("first_tourney_date")
)

# Merge back to main dataframe
df = df.merge(first_tournament, on="player_id", how="left")

In [4]:

# Days of experience (exact)
df["days_of_experience"] = (df["tourney_date"] - df["first_tourney_date"]).dt.days

# Months of experience (approximate)
df["months_of_experience"] = df["days_of_experience"] / 30.44  # average month length

# Years of experience (approximate, adjusted for leap years)
df["years_of_experience"] = df["days_of_experience"] / 365.25

# Career year (integer stage of player’s career: 1, 2, 3, …)
# We floor the years and add 1 so first tournaments = Year 1
df["career_year"] = (df["years_of_experience"].apply(int) + 1)

In [5]:

experience_per_year = (
    df.groupby(["player_id", "year"])["years_of_experience"]
      .max()                     # latest experience in that year
      .reset_index()
      .round(0)
      .rename(columns={"years_of_experience": "experience"})
)

In [6]:
df_merged = df.merge(
    experience_per_year,
    on=["player_id","year"],
    how="left"
)

In [7]:
for n in range(1, 6):
    df_merged[f"round_{n}"] = df_merged[f"round_{n}"].replace(np.nan,"")

In [8]:
encoded_results = []

for row in df_merged.itertuples(index=False):
    row_encoded = []
    outcome = row.match_outcome
    
    for i in range(1, 6):  
        score = getattr(row, f'round_{i}')

        if score == "":
            row_encoded.append("")
            continue

        is_ot = '7-6' in score or '6-7' in score 
        
        if outcome == 1:
            row_encoded.append('OTW' if is_ot else 'W')
        else:
            row_encoded.append('OTL' if is_ot else 'L')
    
    encoded_results.append(row_encoded)

# Assign as separate columns for each round
for i in range(5):
    df_merged[f'encoded_round_{i+1}'] = [row[i] for row in encoded_results]

In [ ]:
for num in range(1, 6):  # loop over your 5 rounds
    col = f'encoded_round_{num}'
    new_col = f'encoded_num_round_{num}'

    df_merged[new_col] = df_merged[col].replace({
        "W": 2,
        "OTW": 1,
        "L": -1,
        "OTL": -2,
        "": np.nan
    }).astype(float)

C:\Users\Golbe\AppData\Local\Temp\ipykernel_15972\397779737.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_merged[new_col] = df_merged[col].replace({
C:\Users\Golbe\AppData\Local\Temp\ipykernel_15972\397779737.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_merged[new_col] = df_merged[col].replace({
C:\Users\Golbe\AppData\Local\Temp\ipykernel_15972\397779737.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `resul

In [10]:
df_merged.to_csv(Path("atp_transformed/2000-2024 XXL.csv"),index=False)